In [235]:
# Imports
import pandas as pd


In [236]:
# Read raw data from excel
df = pd.read_excel('../resources/ConstituentMessagesCallsD4Inbox.xlsx')
df = df.drop(columns=['Phone Number', 'Unnamed: 13', 'Follow Up Response'])
renamed_cols = {
    'Name': 'name',
    'Address': 'address',
    'Affected Address': 'affected_address',
    'Email': 'email_address',
    'Case Number': 'case_number',
    'Date': 'date',
    'Comments': 'constituent_email_1',
    'D4 Response': 'd4_response_1',
    'D4 Staff Member': 'd4_staff_member',
    'Follow Up Response ': 'constituent_email_2',
    'D4 Response.1': 'd4_response_2'
}
df = df.rename(columns=renamed_cols)

# Caese Number
df['case_number'] = df['case_number'].fillna('0')
df['case_number'] = df['case_number'].astype(str)
df['case_number'] = df['case_number'].apply(lambda x: x.split('.')[0])

# Staff Member
scrubbed_df = df.copy()
scrubbed_df['d4_staff_member'] = df['d4_staff_member'].fillna('Staff1')
scrubbed_df['d4_staff_member'] = scrubbed_df['d4_staff_member'].apply(lambda x: x.strip())
scrubbed_df['d4_staff_member'] = scrubbed_df['d4_staff_member'].apply(lambda x: 'Staff1' if 'Ade' in x else 'Staff2')
scrubbed_df['d4_staff_member'].value_counts()

# Scrub name and Email data
scrubbed_df['name'] = df['name'].fillna('Name1')
scrubbed_df['name'] = scrubbed_df['name'].apply(lambda x: x.split(' ')[0])
scrubbed_df['email_address'] = scrubbed_df['name'].apply(lambda x: f'{x.lower()}@email.com')

In [237]:
# Drop rows with no response
rows_to_drop = scrubbed_df[scrubbed_df['d4_response_1'].isna() & scrubbed_df['d4_response_2'].isna()].index
scrubbed_df = scrubbed_df.drop(rows_to_drop)


In [238]:
from numpy import NaN


rows_to_augment = scrubbed_df[scrubbed_df['d4_response_1'].isna() & scrubbed_df['d4_response_2'].notna()].index
for row in rows_to_augment:
    scrubbed_df.loc[row, 'd4_response_1'] = scrubbed_df.loc[row, 'd4_response_2']
    scrubbed_df.loc[row, 'd4_response_2'] = NaN

In [239]:
rows_to_augment_2 = scrubbed_df[scrubbed_df['affected_address'].isna() & scrubbed_df['address'].notna()].index
for row in rows_to_augment_2:
    scrubbed_df.loc[row, 'affected_address'] = scrubbed_df.loc[row, 'address']
    
scrubbed_df = scrubbed_df.drop(['address'], axis=1)

In [241]:
scrubbed_df['d4_response_1'] = scrubbed_df['d4_response_1'].str.replace('KG:', '')
scrubbed_df['d4_response_1'] = scrubbed_df['d4_response_1'].str.replace('KG', '')
scrubbed_df['d4_response_2'] = scrubbed_df['d4_response_2'].str.replace('KG: ', '')
scrubbed_df.head()

,name,affected_address,email_address,case_number,date,constituent_email_1,d4_response_1,d4_staff_member,constituent_email_2,d4_response_2
0,Ron,6864 East Bucknell Place,ron@email.com,0,2024-08-05,The lack of police presence and code enforceme...,"Good morning Ron, \n\nThank you for reaching o...",Staff1,NaN,NaN
1,Renee,Dahlia & High Line intersection,renee@email.com,0,2024-08-19,I would like to know if there is a possible to...,Thank you for reaching out to our office. The...,Staff2,NaN,NaN
2,JW,Happy Canyon & Jasmine St,jw@email.com,0,2024-08-19,Accident took place at Happy Canyon & Jasmine....,"Good morning TJ,\n\nThank you for reaching out...",Staff1,NaN,NaN
3,Clara,Hutchinson Hills,clara@email.com,9578014,2024-08-20,Has many concerns about getting a compost cart...,Thank you for reaching out to our office and f...,Staff2,Clara sent another email back with more questi...,Did not respond as they were the same question...
4,Pauline,NaN,pauline@email.com,0,2024-08-21,At any rate my concern is how will HOA condo u...,"Good afternoon Pauline, \n\nThank you for reac...",Staff1,NaN,NaN


In [242]:
scrubbed_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 57 entries, 0 to 60
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   name                 57 non-null     object        
 1   affected_address     40 non-null     object        
 2   email_address        57 non-null     object        
 3   case_number          57 non-null     object        
 4   date                 57 non-null     datetime64[ns]
 5   constituent_email_1  57 non-null     object        
 6   d4_response_1        57 non-null     object        
 7   d4_staff_member      57 non-null     object        
 8   constituent_email_2  11 non-null     object        
 9   d4_response_2        12 non-null     object        
dtypes: datetime64[ns](1), object(9)
memory usage: 7.0+ KB


In [243]:
# Save the data
scrubbed_df.to_csv('../resources/scrubbed_data.csv', index=False)

## 311 Topics:
https://www.denvergov.org/Online-Services-Hub/Report-an-Issue/issue/new-report

- Homeless
- Graffiti
- Pothole
- Animals
- Weeds
- Neighborhood
- Other
- Snow Removal
- Vehicle
- Parking
- Trees
- Police
- Fireworks
- Dumping
- Micromobility
- Utilities
- Trash

## Other Topics:
- Street Racing
- Traffic Signals
- Policy
- Parks
- HOA